In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

2023-04-29 19:34:08.938234: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 19:34:09.844985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-29 19:34:11.184616: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-29 19:34:11.185027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
from kerastuner import HyperModel
from models import *
class CryptoModel(HyperModel):
    def __init__(self, input_shape, label_width):
        self.input_shape = input_shape
        self.label_width = label_width

    def build(self, hp):
        model_type = hp.Choice('model_type', ['Dense''LSTM', 'LSTM2', 'Conv1D'])
        if model_type == 'Dense':
            model = create_Dense(hp, self.input_shape, self.label_width)
        elif model_type == 'LSTM':
            model = create_LSTM(hp, self.input_shape, self.label_width)
        elif model_type == 'LSTM2':
            model = create_LSTM2(hp, self.input_shape, self.label_width)
        elif model_type == 'Conv1D':
            model = create_Conv1D(hp, self.input_shape, self.label_width) 
        model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                      loss='mse',
                      metrics=['mae'])
        return model

/tmp/ipykernel_148860/1559655559.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel
2023-04-29 08:45:00.060606: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-29 08:45:00.060777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-29 08:45:00.060893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value 

In [5]:

from kerastuner.tuners import RandomSearch

import pandas as pd
import pandas as pd
import datetime
from  window_generator  import WindowGenerator
import pandas as pd
import tensorflow as tf
from binance_data import get_binance_data
from data_processing import train_test   , compile_and_fit , normalize , data_parser  , data_for_prediction_parser
import pandas as pd
import models
import pickle
from datetime import timedelta as td
import window_generator
import config.config as config

from tensorflow.keras.layers import Dense, Dropout, LSTM , GRU , Flatten , BatchNormalization , Conv1D , MaxPooling1D
from tensorflow.keras.models import Sequential



In [4]:


input_shape = config.input_shape
label_width = config.label_width

start_time = "1 Jun 2010"
# Get current date and time
now = datetime.datetime.now()-td(hours=24)
# Format date and time as a string
end_time = now.strftime("%Y-%m-%d %H:%M:%S")

data = get_binance_data ('BTCUSDT',  start_time, end_time )
training_df = data_parser(data)
training_df_norm = normalize(training_df, label_width=config.label_width, window=30)
train_df, val_df, test_df = train_test(training_df_norm[config.variables_used])
wide_window = WindowGenerator(
input_width=config.input_width, label_width=config.label_width, shift=0,
label_columns=config.variables_used, train_df=train_df, val_df=val_df, test_df=test_df)

#hypermodel = CryptoModel(input_shape, label_width)

wide_window = WindowGenerator(
                input_width=config.input_width, label_width=config.label_width, shift=0,
                label_columns=config.variables_used, train_df=train_df, val_df=val_df, test_df=test_df)


In [11]:

def hypermodel(hp):
    model = Sequential()

    # LSTM Layer(s)
    
    #lstm_layers = hp.Int("lstm_layers", min_value=1, max_value=3, step=1)
    units = hp.Int(f"units", min_value=32, max_value=512, step=32)
    units2 = hp.Int(f"units2", min_value=32, max_value=100, step=32)
    dropout_rate = hp.Float(f"dropout", min_value=0.0, max_value=0.5, step=0.1)
    #dropout_rate2 = hp.Float(f"dropout2", min_value=0.0, max_value=0.4, step=0.1)
    model.add( Conv1D( filters=64, kernel_size=3,    input_shape=input_shape))
    model.add(Dropout(dropout_rate)),
    #model.add(MaxPooling1D(pool_size=2)),
    model.add(Dense(units=units2)),
    model.add(Dense(units=32, activation='relu')) 
    #model.add(BatchNormalization())
    # Output Layer
    model.add(Flatten()) 
    model.add(Dense(label_width))
    
    # Compile model
    learning_rate = hp.Choice("learning_rate", values=[1e-3, 1e-4, 1e-5])
    model.compile(optimizer=tf.keras.optimizers.Adam() , metrics=[tf.keras.metrics.MeanAbsoluteError()],
                  loss="mse")
    return model


tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=30,
    executions_per_trial=2,
    directory='tuner_results_4',
    project_name='crypto_hyperopt'
)

tuner.search_space_summary()
tuner.search(wide_window.train,validation_data=wide_window.val, epochs=100,  callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=5)])

best_model = tuner.get_best_models(1)[0]

Trial 30 Complete [00h 01m 36s]
val_loss: 0.01751757226884365

Best val_loss So Far: 0.0006894618854857981
Total elapsed time: 01h 02m 26s
INFO:tensorflow:Oracle triggered exit


In [15]:
tuner.get_best_models(1)[0]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 198, 64)           256       
                                                                 
 dropout (Dropout)           (None, 198, 64)           0         
                                                                 
 dense (Dense)               (None, 198, 64)           4160      
                                                                 
 dense_1 (Dense)             (None, 198, 32)           2080      
                                                                 
 flatten (Flatten)           (None, 6336)              0         
                                                                 
 dense_2 (Dense)             (None, 12)                76044     
                                                                 
Total params: 82,540
Trainable params: 82,540
Non-traina

In [13]:
best_hps=tuner.get_best_hyperparameters()[0]
best_hps.get_config()

{'space': [{'class_name': 'Int',
   'config': {'name': 'units',
    'default': None,
    'conditions': [],
    'min_value': 32,
    'max_value': 512,
    'step': 32,
    'sampling': 'linear'}},
  {'class_name': 'Int',
   'config': {'name': 'units2',
    'default': None,
    'conditions': [],
    'min_value': 32,
    'max_value': 100,
    'step': 32,
    'sampling': 'linear'}},
  {'class_name': 'Float',
   'config': {'name': 'dropout',
    'default': 0.0,
    'conditions': [],
    'min_value': 0.0,
    'max_value': 0.5,
    'step': 0.1,
    'sampling': 'linear'}},
  {'class_name': 'Choice',
   'config': {'name': 'learning_rate',
    'default': 0.001,
    'conditions': [],
    'values': [0.001, 0.0001, 1e-05],
    'ordered': True}}],
 'values': {'units': 352,
  'units2': 64,
  'dropout': 0.0,
  'learning_rate': 0.001}}